In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import os
from IPython.display import display 

import spacy

In [4]:
folder = os.path.join(os.getcwd(), "data", "texts")
files = [os.path.join(root, f) for root, _, files in os.walk(folder) for f in files if f.endswith(".docx")]
        

In [12]:
from src.pipelinelib.querying import Parser, Queryable

nlp = spacy.load("de_core_news_sm", disable=["ner", "parser"])
parser = Parser(nlp=nlp, metadata_path="./data/texts/Kopie von Transkriptionspaare_Daten.xls")

parser.read_from_files(files)

ImportError: Missing optional dependency 'xlrd'. Install xlrd >= 1.0.0 for Excel support Use pip or conda to install xlrd.

In [37]:
display(parser.frame)

,document_id,paragraph_id,sentence_id,couple_id,speaker,gender,is_depressed_group,depressed_person,text
0,0,0,0,27,A,M,True,W,Positive Gefühle.
1,0,1,1,27,B,W,True,W,"Ja, positive Gefühle können bei mir auslösen w..."
2,0,2,2,27,A,M,True,W,"Zuwendung, das ist ja, Zuwendung, wir beide, ..."
3,0,2,3,27,A,M,True,W,"Verteilung der Haushaltsaufgaben, das machen w..."
4,0,2,4,27,A,M,True,W,"Gleiche Vorstellung bei der Kindererziehung, i..."
...,...,...,...,...,...,...,...,...,...
127,9,106,127,87,A,M,False,None,Ich mag dein Lachen.
128,9,106,128,87,A,M,False,None,Hübsch bist du.
129,9,107,129,87,B,W,False,None,Danke.
130,9,108,130,87,A,M,False,None,"Ja… Ich find´s gut, dass du ein Instrument spi..."


In [10]:
from src.utils.querying import Parser, Queryable
from src.pipelinelib.text_body import TextBody

queryable = Queryable.from_parser(parser)

ModuleNotFoundError: No module named 'src.utils.querying'

In [39]:
document_df = queryable.execute(level=TextBody.DOCUMENT)
display(document_df)

=== Queryable is executing on Document, query = '' ===


,document_id,couple_id,is_depressed_group,depressed_person,text
0,0,27,True,W,"Positive Gefühle.. Ja, positive Gefühle können..."
1,1,182,False,None,(unv.. 00:02). Hm?. Du solltest doch nicht nac...
2,2,81,False,None,Ich dachte wir reden darüber wie wir die erste...
3,3,47,True,W,Erzähl mir was.. Also wir müssen die Fragen je...
4,4,58,False,None,Soll ich anfangen?. Ich hab mir was überlegt.....
5,5,29,False,None,"Gut, fangen wir mal an.. Ja, fang du mal an.. ..."
6,6,105,True,W,Jetzt sollen wir uns unterhalten?. Genau.. Al...
7,7,60,True,W,"Wenn´s positiv geht, dann wird es nicht laut, ..."
8,8,138,True,W,(unv.. 2 s.). Am Ammersee.. Dann machen wir es...
9,9,87,False,None,"Wie lang?. 10 Minuten. Okay, dann fang mal an...."


In [40]:
paragraph_df = queryable.execute(level=TextBody.PARAGRAPH)
display(paragraph_df)

=== Queryable is executing on Paragraph, query = '' ===


,document_id,paragraph_id,couple_id,speaker,gender,is_depressed_group,depressed_person,text
0,0,0,27,A,M,True,W,Positive Gefühle.
1,0,1,27,B,W,True,W,"Ja, positive Gefühle können bei mir auslösen w..."
2,0,2,27,A,M,True,W,"Zuwendung, das ist ja, Zuwendung, wir beide, ..."
3,0,3,27,B,W,True,W,"Wir lieben beide Kinder, [A:. Ja] und die lieb..."
4,0,4,27,A,M,True,W,"Es tut gut, und mir tut´s auch gut sie zu beha..."
...,...,...,...,...,...,...,...,...
752,9,103,87,B,W,False,None,"Ja, sich nur über positives zu unterhalten."
753,9,104,87,A,M,False,None,"Ja, stimmt.. Negative Sachen wirft man sich of..."
754,9,106,87,A,M,False,None,"Du bist witzig, du kannst witzig sein.. Ich ma..."
755,9,107,87,B,W,False,None,Danke.


In [41]:
sentence_df = queryable.execute(level=TextBody.SENTENCE)
display(sentence_df)

queryable.execute()

=== Queryable is executing on Sentence, query = '' ===


,document_id,paragraph_id,sentence_id,couple_id,speaker,gender,is_depressed_group,depressed_person,text
0,0,0,0,27,A,M,True,W,Positive Gefühle.
1,0,1,1,27,B,W,True,W,"Ja, positive Gefühle können bei mir auslösen w..."
2,0,2,2,27,A,M,True,W,"Zuwendung, das ist ja, Zuwendung, wir beide, ..."
3,0,2,3,27,A,M,True,W,"Verteilung der Haushaltsaufgaben, das machen w..."
4,0,2,4,27,A,M,True,W,"Gleiche Vorstellung bei der Kindererziehung, i..."
...,...,...,...,...,...,...,...,...,...
127,9,106,127,87,A,M,False,None,Ich mag dein Lachen.
128,9,106,128,87,A,M,False,None,Hübsch bist du.
129,9,107,129,87,B,W,False,None,Danke.
130,9,108,130,87,A,M,False,None,"Ja… Ich find´s gut, dass du ein Instrument spi..."


TypeError: execute() missing 1 required positional argument: 'level'

In [32]:
pgs = queryable.by_couple_id(couple_id=27) \
    .is_depressed(d=False) \
    .execute(level=TextBody.DOCUMENT)
display(pgs)

=== Queryable is executing on Document, query = '(couple_id == 27) and (is_depressed_group == False)' ===


,document_id,couple_id,is_depressed_group,depressed_person,text
